In [36]:
# For webpage Scrapper
import pandas as pd 
import requests
from bs4 import BeautifulSoup

# For Google Sheet functions
import pickle
import os.path
import os
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

#-----------------------
# WEB SCRAPPER - CASH.CH
#-----------------------
# Cash website link - request
wholeHtmlPage = requests.get("https://www.cash.ch/#frontpage-section-4")

# create beautifulsoup object
pSoup = BeautifulSoup(wholeHtmlPage.content, "html.parser")

#nLimit = 13
nLimit = 13

# Index Overview list only first 10 entries = indixes and commodities
#pDescFields = pSoup.find_all("td", {"data-field-entry": "value"}, limit=nLimit)
#pValues = pSoup.find_all("td",{"class": "lval"}, limit=nLimit)
#pDiffValues = pSoup.find_all("td",{"class": "i-net-vperpr-v-pr"}, limit=nLimit)

pDescFields = pSoup.find_all("a", {"class": "cash-link"}, limit=nLimit)
pValues = pSoup.find_all("td",{"class": "lval-norm"}, limit=nLimit)
pDiffValues = pSoup.find_all("td",{"class": "nc2-pr-norm"}, limit=nLimit)

# populate list-object with the 13 entries
pIndexNameList = [desc.text for desc in pDescFields]
pIndexValueList = [value.text for value in pValues]
pIndexDiffValueList = [diff.text for diff in pDiffValues]

# Pandas - Create data frame (table)
df_export_Indices = pd.DataFrame(
   {
    'Index_Name': pIndexNameList,
    'Index_Value': pIndexValueList,
    'Index_Diff': pIndexDiffValueList

    }
)

print(df_export_Indices)
# -------------------------------------------------
# File evaluation - Roberto Schlatter
# 15-10-2020
#-------------------------------------------------
   
rootdir = '/Users/SCHLARO4/1_PRIVATE/Prog/Private/'

sFilename = 'transactions-from-01012010-to-07102020.csv'

df_transactions = pd.read_csv(sFilename, sep=';', encoding= 'unicode_escape')

lst_header = ['Transaktionen', 'Name']

#----------
# DIVIDENDE
#----------
# Filter table to show only columns "Transaktionen", "Name"
df_dividende = df_transactions[df_transactions['Transaktionen'].isin(['Dividende'])]

df_dividende['Nettobetrag in der Währung des Kontos'] = pd.to_numeric(df_dividende['Nettobetrag in der Währung des Kontos'])

# Final dataframe for dividends
df_export_dividende = df_dividende.groupby(lst_header)['Nettobetrag in der Währung des Kontos'].sum()
print(df_export_dividende)

df_export_dividende.to_excel('nettodividende.xlsx')
#print("Dividende: ", df_export_dividende)

#-------
#KOSTEN
#-------
# KAUF & VERKAUF Kosten
df_kosten_total = df_transactions[df_transactions['Transaktionen'].isin(['Kauf', 'Verkauf'])]

# Final dataframe for costs
df_export_kosten_total = df_kosten_total.groupby(lst_header)['Kosten'].sum()

#print(df_export_kosten_total)

df_export_kosten_total.to_excel('kosten.xlsx')
ß

#**********************************************
# Google Sheets Storing - STOCK-MONITOR
#**********************************************

#change this by your sheet ID_ STOCK-MONITOR
#SAMPLE_SPREADSHEET_ID_input = '1RX2lJwZYWUO_sJUME1M9CsMFkGQNEqMCiY2Y6vCZjF0'

#change the range if needed: SUMMARY
#SAMPLE_RANGE_NAME = 'SUMMARY!A25:AA100'
def storeDataToGoogleSheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME, df_gold, sOrientation):
    def Create_Service(client_secret_file, api_service_name, api_version, *scopes):
        global service
        SCOPES = [scope for scope in scopes[0]]
        #print(SCOPES)
        cred = None

        if os.path.exists('token_write.pickle'):
            with open('token_write.pickle', 'rb') as token:
                cred = pickle.load(token)

        if not cred or not cred.valid:
            if cred and cred.expired and cred.refresh_token:
                cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
                cred = flow.run_local_server()

            with open('token_write.pickle', 'wb') as token:
                pickle.dump(cred, token)

        try:
            service = build(api_service_name, api_version, credentials=cred)
            print(api_service_name, 'service created successfully')
            #return service
        except Exception as e:
            print(e)
            #return None
        
    # change 'my_json_file.json' by your downloaded JSON file: my_credentials.json
    Create_Service('my_credentials.json', 'sheets', 'v4',['https://www.googleapis.com/auth/spreadsheets'])
    
    def Export_Data_To_Sheets():
        response_date = service.spreadsheets().values().update(
            spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
            valueInputOption='RAW',
            range=SAMPLE_RANGE_NAME,
            body=dict(
                #Orientation: ROWS oder COLUMNS
                majorDimension=sOrientation,
                values=df_gold.T.reset_index().T.values.tolist())
        ).execute()
        print('Sheet successfully Updated')

    Export_Data_To_Sheets()
    print(df_export_dividende)
 #call storeDataToGoogleSheet function
storeDataToGoogleSheet('1RX2lJwZYWUO_sJUME1M9CsMFkGQNEqMCiY2Y6vCZjF0', 'SUMMARY!B37', df_export_Indices,'ROWS')
storeDataToGoogleSheet('1RX2lJwZYWUO_sJUME1M9CsMFkGQNEqMCiY2Y6vCZjF0', 'SUMMARY!B54', df_export_dividende, 'COLUMNS')
storeDataToGoogleSheet('1RX2lJwZYWUO_sJUME1M9CsMFkGQNEqMCiY2Y6vCZjF0', 'SUMMARY!B79', df_export_kosten_total, 'COLUMNS')

         Index_Name Index_Value Index_Diff
0               SMI   10'507.06     -0.24%
1               SPI   13'035.93     -0.15%
2     EURO STOXX 50    3'432.79     -1.00%
3               DAX   13'055.58     -1.22%
4         DOW JONES   29'226.76     -0.58%
5            NASDAQ   11'814.25     +0.24%
6            NIKKEI   25'520.88     +0.68%
7           EUR/CHF      1.0793     -0.07%
8           USD/CHF      0.9140     -0.34%
9        BUNDESOBLI      -0.424    -12.47%
10  Ölpreis (Brent)       44.38     +1.42%
11        Gold 1 Uz    1'880.50     +0.78%
12   Bitcoin in USD   16'047.80     +2.20%
Transaktionen  Name                          
Dividende      AMERICAN EXPRESS ORD                27.39
               AMS                                 38.08
               APPLE ORD                          161.00
               ISHARES 20 PLS YEAR TREASURY B     152.56
               MICROSOFT ORD                       42.12
               NVIDIA ORD                          12.82
          